In [1]:
import dash_cytoscape as cyto
import graph_tool as gt
import graph_tool.util as gu
import os
import pickle

from dash import Dash, dcc, html, Input, Output, State
from graph_tool import GraphView
from jupyter_dash import JupyterDash

In [2]:
def get_network(node, graph_type):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    g = gr
    v = gu.find_vertex(g, g.vp.name, node)[0]
    nodes = []
    edges= []
    vfilt = g.new_vertex_property('bool')
    vfilt[v] = True
    nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v]}, 'classes':'green'})
    for e, w in zip(v.in_edges(), v.in_neighbors()):
        vfilt[w] = True
        nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w]}, 'classes': 'red triangle'})
        if graph_type != "allCelltypeGraph.gt":
            edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.standDnase1Log2[e]}})
        else:
            edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.score[e]}})
    cyto_list = nodes+edges
    return cyto_list

In [3]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("REMnet", style={"textAlign":"center", "text-decoration":"underline", "fontSize":40}),
    html.Div(id='dd-output-container'),
    html.Div([
            "Choose algorithm",
            dcc.Dropdown(
                id='dropdown-algo',
                value=[],
                clearable=False,
                options=[
                    {'label': name, 'value': name}
                    for name in ["betweenness", "closeness", "trust"]
                ]
            )
        ], style={'height': '90px'}
        ),
    html.Div([
        html.Div([
            "Choose the celltype",
            dcc.Dropdown(
                id='dropdown-graph',
                value=os.listdir('../data/graph-tool/')[0],
                clearable=False,
                options=[
                    {'label': name, 'value': name}
                    for name in os.listdir('../data/graph-tool/')
                ]
            )
        ], style={'display': 'block'}
        ),
        html.Div([
            "Choose the gen you would like to inspect",
            dcc.Dropdown(
            id='dropdown-gen',
            value = 'ENSG00000278267',
            clearable=True,
            options=[]
            )
        ],style={'display': 'block'}
        )
    ], style={'display': 'inline-block', 'vertical-align': 'top', 'width': '34%'}
    ),
#     dcc.Dropdown(
#         id='dropdown-rem',
#         value=rem_list[0],
#         clearable=True,
#         options=[
#             {'label': name, 'value': name}
#             for name in rem_list
#         ]
#     ),
    html.Div([
        cyto.Cytoscape(
            id='network-graph',
            style={'width': '100%', 'height': '700px'},
            layout={'name': 'circle'},
            elements=[],
            stylesheet=[
    #             # Group selectors
                {
                    'selector': 'node',
                    'style': {
                        'content': 'data(label)'
                    }
                },
                {
                    'selector': 'edge',
                    'style': {
                        'content': 'data(label)'
                    }
                },

                # Class selectors
                {
                    'selector': '.red',
                    'style': {
                        'background-color': 'red',
                        'line-color': 'red'
                    }
                },
                {
                    'selector': '.green',
                    'style': {
                        'background-color': 'green',
                        'line-color': 'green'
                    }
                },
                {
                    'selector': '.triangle',
                    'style': {
                        'shape': 'triangle'
                    }
                }
            ]
        )
    ], style={"border":"2px black solid", 'vertical-align': 'top', 'display': 'inline-block', 'width': '65%', 'height': '800px', 'float': 'right'}
    ),

    # dcc.Store stores the intermediate value
    dcc.Store(id='store-data')
])

@app.callback(Output('dd-output-container', 'children'),
              Output('dropdown-gen', 'options'),
              Output('store-data', 'data'),
              Input('dropdown-graph', 'value'))
def update_file(value):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    gr = gt.load_graph(f"../data/graph-tool/{value}")
    gen_list = [{'label': name, 'value': name} for name in [a for a in list(gr.vp.name) if a.startswith("E")]]
    return f'You have selected {value}', gen_list, value

@app.callback(Output('network-graph', 'elements'),
              Input('dropdown-gen', 'value'),
              Input('store-data', 'data'),
              State('network-graph', 'elements'))
def update_elements(gen, name, elements):
    return get_network(gen, name)



# @app.callback(Output('store-data', 'data'),
#               Input('dropdown-graph', 'value'))
# def load_graph(value):
#     return [[gt.load_graph(f'../data/graph-tool/{value}')]]

# @app.callback(Output('network-graph', 'elements'),
#               Input('dropdown-rem', 'value'),
#               State('network-graph', 'elements'))
# def update_elements(value, elements):
#     return get_network(value)

# app.run_server(host='0.0.0.0', port=8050, debug=False)
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [ ]:
import matplotlib.pyplot as plt

weights = [g.ep.standDnase1Log2[e] for e in g.edges()]
plt.hist(weights)
plt.xlabel('weight')
# plt.show()

In [8]:
graph_pickle_safe = open("graph_pickle", "wb")
gen_list = [{'label': name, 'value': name} for name in [a for a in list(gr.vp.name) if a.startswith("E")]]
pickle.dump(gr, graph_pickle_safe)
graph_pickle_safe.close()
sub = GraphView(g, vfilt)


AttributeError: 'super' object has no attribute 'standDnase1Log2'